In [4]:
import numpy as np
import torchio as tio
from aicsimageio import AICSImage
from functools import partial

# only for visualization 
# follow the instruction to make proper installation: https://pypi.org/project/nbvv/
import nbvv

In [5]:
# make a customizaed reading wrapper for aicsimageio
def aicsimageio_wrapper(fn, **kwargs):
    img = AICSImage(fn).reader.get_image_dask_data(**kwargs)
    img_data = tio.data.io.check_uint_to_int(img.compute())
    return img_data, np.eye(4)

In [3]:
# specify readering parameters
# in real situation, we usually read such parameters from a configuration file (e.g., json or yaml)
my_reader_params = {
    "dimension_order_out": "CZYX",
    "C": [1,2],
    "T": 0
}

In [14]:
# define the actual reader according to user-specified reading parameters
bio_reader = partial(aicsimageio_wrapper, **my_reader_params)

In [15]:
filepath = "./sample_data/multi_channel_3D_sample.tiff"
my_subject = tio.Subject(
    input=tio.ScalarImage(filepath, reader=bio_reader),
    label = "Normal"
)

In [16]:
# note: the loading step won't be executed until the data is actually needed
img = my_subject["input"][tio.DATA]

In [17]:
img.shape

torch.Size([2, 70, 200, 200])

In [20]:
nbvv.volshow(img.numpy(), spacing=[1.0, 1.0, 3.0], channel_names=["nuclear_envolope", "DNA dye"])

VolumeWidget(dimensions={'tile_width': 200, 'tile_height': 200, 'rows': 8, 'cols': 9, 'atlas_width': 1800, 'at…